<img src="https://drive.google.com/uc?export=view&id=1WNLKH10YpQNNk9eeRIyYLwGkxNbNp-Mm" width="100%">

# Modelos de Tópicos
---

En este notebook veremos una introducción a los modelos de tópicos desde _Python_. Comenzamos importando las librerías necesarias:

In [ ]:
!pip install unidecode

In [ ]:
import re
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
from IPython.display import display

## **1. Motivación**
---

Un modelo de tópicos es una técnica de procesamiento del lenguaje natural que se utiliza para analizar y entender el contenido de un corpus. El objetivo es identificar los temas o temas subyacentes que se discuten en esos documentos. Es una forma de agrupar y resumir automáticamente información contenida en un gran número de textos. Por ejemplo, en la siguiente figura vemos un ejemplo aplicado sobre un documento científico relacionado con bioinformática. Podemos ver que se encuentran algunos tópicos que podemos relacionar con genética (tópico 1), biología (tópico 2), computación (tópico 3). De la misma forma, también podemos ver qué tan importante es cada tópico para el documento en específico. Más adelante veremos cómo podemos interpretar estos modelos.

<img src="https://drive.google.com/uc?export=view&id=11ARBlZx2qpXjqmMzpohe28iOLD-M2MFn" width="80%">

Los modelos de tópicos utilizan una variedad de algoritmos para analizar el contenido de los documentos y extraer los temas subyacentes. Una vez que se han identificado los temas, se pueden utilizar para clasificar automáticamente nuevos documentos, hacer recomendaciones, generar resúmenes, entre otras aplicaciones.

En la creación de un modelo de tópicos se suelen seguir los siguientes pasos:

- **Selección y limpieza de los datos**: se recopilan y se limpian los datos a analizar para quedarnos con solo la información relevante.
- **Tokenización del texto**: se divide el texto en palabras o frases para poder analizarlas de manera individual.
- **Eliminación de palabras irrelevantes**: se eliminan las palabras que no aportan información relevante para el análisis, como artículos o preposiciones.
- **Creación del diccionario**: se crea un diccionario de todas las palabras relevantes y se les asigna un valor numérico.
- **Creación del modelo**: se utiliza un algoritmo específico para analizar los datos y asignar un tópico a cada palabra.

Existen diferentes algoritmos y técnicas que se utilizan para crear modelos de tópicos, los más populares son: **Latent Semantic Analysis (LSA)** y **Latent Dirichlet Allocation (LDA)**. Veremos cómo podemos entrenar este tipo de modelos.

## **2. Carga de Datos**
---

Como los modelos de tópicos se entrenan de forma no supervisada, lo único que necesitamos para entrenarlos es un conjunto de textos (también es posible utilizar un único documento si tokenizamos por oraciones).

En este caso usaremos el conjunto de datos **20 newsgroups**, la cual trata de una colección de alrededor de 11000 mensajes de noticias en 20 diferentes categorías o grupos de noticias. Los 20 grupos de noticias incluyen temas como ciencias políticas, religión, deportes, tecnología y ciencia. Cada mensaje incluye el encabezado del correo electrónico, el cuerpo del mensaje y el remitente. Los mensajes están en inglés y se encuentran en formato de texto plano.

Fue recolectado por el equipo de investigación de aprendizaje automático de la Universidad de Massachusetts en los años 90 y se ha utilizado ampliamente como una de las principales fuentes de datos para evaluar los algoritmos de aprendizaje automático y procesamiento del lenguaje natural. Este conjunto de datos es ampliamente utilizado en la investigación y la educación, ya que proporciona una variedad de textos en diferentes temas y una estructura de datos fácil de trabajar. Se utiliza para evaluar los algoritmos de clasificación de texto, extracción de características, agrupamiento de texto y otros problemas relacionados con el procesamiento del lenguaje natural.

Este conjunto de datos lo podemos cargar directamente desde `sklearn`, importamos la función correspondiente:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

Procedemos a cargar el conjunto de datos eliminando algunos artefactos relacionados a etiquetas HTML:

In [ ]:
dataset = fetch_20newsgroups(remove=("headers", "footer", "quotes"))
display(dataset)

Ahora, extraemos el texto y los posibles tipos de noticias:

In [ ]:
corpus = dataset.data
labels = dataset.target_names
print(len(corpus))
print(len(labels))

Veamos un documento al azar del conjunto de datos:

In [ ]:
idx = np.random.randint(len(corpus))
doc = corpus[idx]
print(doc)

Ahora, veamos los posibles tipos de noticias en el corpus:

In [ ]:
display(labels)

Estos corresponden a:

- `alt.atheism`: Noticias relacionadas con el ateísmo.
- `comp.graphics`: Noticias relacionadas con la computación gráfica.
- `comp.os.ms-windows.misc`: Noticias relacionadas con el sistema operativo Microsoft Windows.
- `comp.sys.ibm.pc.hardware`: Noticias relacionadas con el hardware de la computadora IBM PC.
- `comp.sys.mac.hardware`: Noticias relacionadas con el hardware de la computadora Macintosh.
- `comp.windows.x`: Noticias relacionadas con la interfaz gráfica de usuario de Windows X.
- `misc.forsale`: Noticias relacionadas con la venta de artículos.
- `rec.autos`: Noticias relacionadas con los automóviles.
- `rec.motorcycles`: Noticias relacionadas con las motocicletas.
- `rec.sport.baseball`: Noticias relacionadas con el béisbol.
- `rec.sport.hockey`: Noticias relacionadas con el hockey sobre hielo.
- `sci.crypt`: Noticias relacionadas con la criptografía.
- `sci.electronics`: Noticias relacionadas con la electrónica.
- `sci.med`: Noticias relacionadas con la medicina.
- `sci.space`: Noticias relacionadas con el espacio.
- `soc.religion.christian`: Noticias relacionadas con la religión cristiana.
- `talk.politics.guns`: Noticias relacionadas con la política de armas.
- `talk.politics.mideast`: Noticias relacionadas con la política del Medio Oriente.
- `talk.politics.misc`: Noticias relacionadas con la política en general.
- `talk.religion.misc`: Noticias relacionadas con la religión en general.

Es importante tener en cuenta que algunos de estos temas pueden ser amplios y pueden incluir subtemas específicos.

Ahora, vamos a definir una función para preprocesar los textos. Primero definimos un pipeline de `spacy` (para tokenizar, eliminar stopwords y etiquetar partes del discurso - POS tagging).

In [ ]:
nlp = spacy.load('en_core_web_sm',
                 exclude=['parser', 'senter', 'lemmatizer', 'ner'])
nlp.component_names

Ahora, definimos la función de preprocesamiento:

In [ ]:
def preprocess(text):
    no_chars = re.sub(r"[^a-z ]", " ", text) # eliminamos caracteres especiales
    doc = nlp(no_chars) # creamos un documento de spacy
    no_stops = " ".join(
        token.text
        for token in filter(
            lambda token: not token.is_stop  # eliminamos stopwords
                          and len(token) > 3 and len(token) < 24 # eliminamos palabras por longitud
                          and token.pos_ in ['NOUN', 'PROPN'], # dejamos solo sustantivos o nombres propios
            doc,
            )
        )
    norm_text = unidecode(no_stops.lower()) # normalizamos el texto
    no_spaces = re.sub(r"\s+", " ", norm_text) # eliminamos espacios duplicados
    return no_spaces.strip()

Preprocesamos el corpus:

In [ ]:
prep_corpus = list(map(preprocess, corpus))
display(prep_corpus[:5])

## **3. Extracción de Características**
---

Los modelos de tópicos por lo general se aplican sobre representaciones basadas en bolsas de palabras. Más adelante hablaremos un poco más en detalle de cada modelo. Por el momento nos concentraremos en entrenar vectorizadores para conteo de palabras y para TF-IDF. Comenzamos importándolos:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Entrenamos los vectorizadores para utilizar únicamente los 500 términos más frecuentes:

In [ ]:
bow = CountVectorizer(max_features=500).fit(prep_corpus)
tfidf = TfidfVectorizer(max_features=500).fit(prep_corpus)

Ahora, extraemos las representaciones:

In [ ]:
features_bow = bow.transform(corpus)
features_tfidf = tfidf.transform(corpus)

También extraemos el vocabulario de cada tokenizador:

In [ ]:
vocab_bow = bow.get_feature_names_out()
vocab_tfidf = tfidf.get_feature_names_out()

## **4. Latent Semantic Analysis**
---

El Análisis Semántico Latente (Latent Semantic Analysis o LSA) es una técnica de procesamiento del lenguaje natural que se utiliza para analizar y comprender el contenido de un conjunto de documentos. El objetivo es identificar los temas subyacentes que se discuten en esos documentos y establecer relaciones semánticas entre las palabras y frases. Es una forma de agrupar y resumir automáticamente la información contenida en un gran número de textos.

LSA se basa en la idea de que las palabras que aparecen juntas en un documento tienden a tener un significado relacionado. Por ejemplo, las palabras "perro" y "ladrar" probablemente están relacionadas semánticamente, ya que un perro ladra. El objetivo de LSA es encontrar estas relaciones semánticas ocultas entre las palabras en un conjunto de documentos.

Para llevar a cabo el análisis, se crea una matriz documento-término (bolsa de palabras) que representa los conteos de los términos (palabras) en los documentos. Cada celda en la matriz contiene el número de veces que aparece un término en un documento. Esta matriz se llama matriz de contenido. A continuación, se aplica una técnica de reducción de dimensionalidad, como la descomposición en valores singulares (SVD), para reducir la dimensionalidad de la matriz de contenido.

<img src="https://drive.google.com/uc?export=view&id=1f-GEwcEPXMFLETqhq2QRVEZg_whsGZAb" width="80%">

SVD divide la matriz en tres componentes: una matriz documento-tópico $\mathbf{U}$, una matriz tópico-término $\mathbf{V}$ y una matriz de importancias $\mathbf{S}$. Estas tres matrices tienen su interpretación:

- **Matriz documento-tópico**: se trata de una matriz de características (funciona como _embedding_) que representa cada documento como una composición de varios tópicos.
- **Matriz de importancias**: esta matriz muestra qué tan importante es cada tópico para todo el corpus.
- **Matriz tópico-término**: se trata de una matriz que muestra qué tan importante es cada término para un tema.

### **4.1. Implementación**
---

Para implementar este modelo vamos a usar la descomposición de `sklearn`:

In [ ]:
from sklearn.decomposition import TruncatedSVD

En **LSA** se utiliza el enfoque **TF-IDF** en lugar de contar simplemente el número de veces que aparece un término en un documento, por varias razones:

- **Manejo de términos irrelevantes**: algunos términos comunes aparecen con mucha frecuencia en todos los documentos y no aportan información relevante para el análisis. TF-IDF tiene en cuenta estos términos y los penaliza.
- **Importancia relativa de los términos**: algunos términos son más importantes que otros en un documento. TF-IDF tiene en cuenta no solo la frecuencia de un término en un documento, sino también su frecuencia en todos los documentos del corpus.
- **Discriminación de términos**: LSA utiliza SVD para reducir la dimensionalidad de la matriz de representación, y algunos términos son necesarios para discriminar entre los documentos. TF-IDF ayuda a identificar estos términos importantes y a incluirlos en la representación.
- **Mejora de la precisión**: al utilizar TF-IDF en lugar de conteos simples, se obtiene una representación continua con una magnitud controlada, esto facilita la precisión numérica en el algoritmo SVD.

Los hiperparámetros de TruncatedSVD de `sklearn` para implementar LSA son:

- `n_components`: Es el número de componentes o dimensiones deseadas en la reducción de dimensionalidad. Este valor representa el número de tópicos o temas que deseamos encontrar.
- `algorithm`: El algoritmo utilizado para calcular la descomposición SVD. Los valores posibles son "arpack" y "randomized". El valor predeterminado es "randomized" que es más rápido para matrices grandes.
- `n_iter`: Número de iteraciones para el algoritmo "randomized". El valor predeterminado es 5.
- `random_state`: Es la semilla utilizada para la inicialización del generador de números aleatorios.

Es importante tener en cuenta que, al elegir los valores para estos hiperparámetros, debe tener en cuenta el tamaño de su conjunto de datos y los requisitos de precisión y tiempo de ejecución. En este caso usaremos los siguientes valores:

In [ ]:
lsa = TruncatedSVD(
        n_components = 20,
        algorithm = "randomized",
        random_state = 42,
        n_iter = 100
        ).fit(features_tfidf)

### **4.2. Tópicos por Documento**
---

Comenzaremos extrayendo una matriz de representación de los textos como si fuera un _embedding_, para esto, usaremos el método `transform` del modelo:

In [ ]:
features_lsa = lsa.transform(features_tfidf)
print(features_lsa.shape)

Como podemos ver, cada documento fue mapeado a una representación de tamaño 20. No obstante, se trata de una representación que resume todo el vocabulario en únicamente 20 temas.

Podemos visualizar la representación de un documento específico:

In [ ]:
doc_id = 0
doc_features = features_lsa[doc_id]
fig, ax = plt.subplots()
ax.bar(np.arange(doc_features.size), doc_features);
ax.set_xlabel("Tópico");
ax.set_ylabel("Valor");
ax.set_xticks(np.arange(doc_features.size));
fig.show()

LSA se caracteriza por ser el modelo de tópicos de menor costo computacional (se entrena bastante rápido). No obstante, no se puede interpretar tan fácilmente, ya que utiliza valores negativos y en una escala que depende de los datos originales.

En ese caso, podemos ver la importancia de cada tópico en un documento específico si miramos la magnitud de su representación, es decir, el valor absoluto:

In [ ]:
doc_id = 0
doc_features = np.abs(features_lsa[doc_id])
fig, ax = plt.subplots()
ax.bar(np.arange(doc_features.size), doc_features)
ax.set_xlabel("Tópico")
ax.set_ylabel("Valor")
ax.set_xticks(np.arange(doc_features.size));
fig.show()

Esta representación suele ser muy usada como _embedding_ de documentos. Veamos un ejemplo de similitud semántica con el siguiente documento:

In [ ]:
doc_id = 1
print(corpus[doc_id])

Como podemos ver, es un texto que habla de hardware de computadores. Vamos a evaluar la similitud coseno con la representación LSA de cada uno de los documentos para determinar los más similares. Importamos la función para la similitud:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

Evaluamos la similitud:

In [ ]:
sim = cosine_similarity(features_lsa[doc_id, np.newaxis], features_lsa).flatten()
print(sim.shape)

Ahora, creamos un `DataFrame` para encontrar el top de los documentos más similares:

In [ ]:
sims = pd.DataFrame(data={"text": corpus, "sim": sim})
top5 = (
        sims
        .sort_values(by="sim", ascending=False)
        .head(6)
        )

Veamos los documentos más relevantes:

In [ ]:
for doc in top5.iloc[1:, 0]:
    print("=" * 50)
    print(doc)

Como podemos ver, estos documentos también hablan sobre hardware.

### **4.3. Términos por Tópico**
---

La matriz tópico-término nos ofrece información muy valiosa para la interpretación de los tópicos. En especial, nos permite saber cuáles son las palabras más importantes en los temas que encontró el modelo.

Podemos acceder a esta matriz por medio del atributo `components_` del modelo:

In [ ]:
components = lsa.components_
print(components.shape)

Como podemos ver, nos muestra la importancia de cada una de las 500 palabras del vocabulario para cada uno de los 20 tópicos. Veamos el top 15 de las palabras más importantes por cada tópico:

In [ ]:
# Iteramos sobre cada tópico
for i, comp in enumerate(components):
    # Juntamos los términos con cada uno de los valores en la matriz V
    terms_comp = zip(vocab_tfidf, np.abs(comp))
    # Ordenamos los términos de acuerdo al resultado de LSA
    sorted_terms = sorted(
            terms_comp,
            key=lambda x: x[1],
            reverse=True
            )[:15]
    # Mostramos los términos más importantes en cada tópico
    print(
            "Tópico {}: {}".format(
                i,
                " ".join(list(map(lambda x:x[0], sorted_terms)))
                )
            )

Con esto podemos ver algunos tópicos clave como:

- **Tópico 4**: deportes.
- **Tópico 7**: hardware.

No obstante, muchos de los otros tópicos parecen mezclar distintos temas (baja coherencia).

Este tipo de análisis se puede ver de mejor forma con una visualización de nubes de palabras. Importamos la librería:

In [ ]:
from wordcloud import WordCloud

Vamos a generar una nube de palabras ponderada con la importancia de cada término en un tópico:

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15, 7))
cont = 0
for i in range(4):
    for j in range(5):
        ax = axes[i, j]
        freqs = {
            term: abs(float(importance))
            for term, importance in zip(vocab_tfidf, components[cont])
        }
        wc = WordCloud(background_color="white").generate_from_frequencies(freqs)
        ax.imshow(wc)
        ax.axis("off")
        ax.set_title(f"Tópico {cont}")
        cont += 1
fig.show()

### **4.4. Importancia de Tópicos**
---

También podemos ver qué tan importante es cada tópico dentro del corpus con la matriz de importancias de tópico. Se trata de una matriz diagonal con un valor que puede estar normalizado; se puede extraer por medio del atributo `explained_variance_ratio_` del modelo:

In [ ]:
topic_importances = lsa.explained_variance_ratio_
print(topic_importances.shape)

El resultado es directamente la diagonal de la matriz. Podemos generar una gráfica en forma de diagrama de barras para ver estas importancias:

In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(topic_importances.size), topic_importances)
ax.set_xticks(np.arange(topic_importances.size))
ax.set_xlabel("Tópico")
ax.set_ylabel("Importancia")
fig.show()

## **5. Latent Dirichlet Allocation**
---

El modelo de tópicos Latent Dirichlet Allocation (LDA) es un algoritmo de aprendizaje automático no supervisado utilizado para identificar los temas subyacentes que se discuten en un conjunto de documentos. LDA asume que cada documento está compuesto por una mezcla de tópicos y que cada tópico está compuesto por una mezcla de palabras.

<img src="https://drive.google.com/uc?export=view&id=1j1neXJH2PhQiS5Nz1Mi3F8-BY312BvWg" width="80%">

LDA utiliza una distribución de probabilidad generativa para modelar cómo se generan los documentos. La idea es que, dado un conjunto de tópicos, se asigna un tópico a cada palabra en un documento de forma aleatoria. Luego, se utiliza la distribución de probabilidad para generar nuevos documentos que se ajusten al conjunto de datos original.

La implementación de LDA requiere especificar el número de tópicos a generar, el número de palabras en cada tópico y las distribuciones de probabilidad para cada uno. Luego, se utiliza un algoritmo de optimización para ajustar estos parámetros al conjunto de datos.

LDA es un modelo probabilístico que se basa en la teoría Bayesiana, pero más allá de entender los detalles matemáticos, en este caso nos interesa ver su aplicabilidad práctica. En LDA tenemos los dos siguientes parámetros:

- $\alpha$: es un parámetro de la distribución Dirichlet que representa la densidad documento-tópico. Entre más grande sea el valor de $\alpha$, cada documento estará conformado por más tópicos.
- $\beta$: es el parámetro _a priori_ que representa la densidad tópico-término. Con un valor grande de $\beta$ los tópicos se conforman de un mayor número de palabras.

LDA es un modelo muy similar a LSA, con la diferencia de que todas las representaciones y elementos internos son probabilidades, lo que facilita su interpretación. No obstante, LDA es un modelo con mayor costo computacional (puede tardar más tiempo en entrenar).

### **5.1. Implementación**
---

Para implementar LDA utilizaremos el modelo de descomposición desde `sklearn`:

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

En **LDA** se utilizan **conteos de palabras** en lugar de **TF-IDF**, debido a que el modelo de tópicos LDA es una distribución generativa, donde se asume que cada documento es una mezcla de tópicos y cada tópico es una mezcla de palabras.

Los conteos de palabras representan la frecuencia de una palabra en un documento, lo que es importante para LDA, ya que asume que las palabras con una mayor frecuencia en un documento son más probables de ser asociadas con el tópico del documento. En cambio, TF-IDF se utiliza para representar la importancia relativa de una palabra en un documento en comparación con el conjunto de documentos, lo que no es necesario para LDA, ya que se asume que cada documento es independiente.

Además, LDA se basa en la idea de que los tópicos son distribuciones de probabilidad sobre las palabras, y los conteos de palabras son adecuados para representar esta distribución. Al utilizar conteos de palabras en lugar de TF-IDF, se pueden calcular las probabilidades de las palabras en cada tópico y en cada documento, lo que es esencial para el funcionamiento del modelo.

Procedemos a entrenar el modelo con la representación de bolsa de palabras. Tiene los siguientes hiperparámetros:

- `n_components`: número de tópicos.
- `doc_topic_prior`: valor $\alpha$.
- `topic_word_prior`: valor $\beta$.
- `random_state`: semilla de números aleatorios.

In [ ]:
lda = LatentDirichletAllocation(
    n_components=20,
    doc_topic_prior=1 / 20,
    topic_word_prior=1 / 20,
    random_state=42,
    ).fit(features_bow)

### **5.2. Tópicos por Documento**
---

Comenzamos extrayendo la matriz documento-tópico con el método `transform` del modelo:

In [ ]:
features_lda = lda.transform(features_bow)
print(features_lda.shape)

Podemos visualizar la representación de un documento:

In [ ]:
doc_id = 0
doc_features = features_lda[doc_id]
fig, ax = plt.subplots()
ax.bar(np.arange(doc_features.size), doc_features);
ax.set_xlabel("Tópico");
ax.set_ylabel("Valor");
ax.set_xticks(np.arange(doc_features.size));
fig.show()

Como podemos ver, se trata de valores entre 0 y 1 (son probabilidades) y la suma es 1:

In [ ]:
print(np.round(doc_features.sum(),4))

Al igual que en LSA, el modelo LDA también captura relaciones semánticas entre documentos. Veamos un ejemplo con la similitud coseno, primero seleccionamos un documento:

In [ ]:
doc_id = 1
print(corpus[doc_id])

Calculamos la similitud:

In [ ]:
sim = cosine_similarity(features_lda[doc_id, np.newaxis], features_lda).flatten()
print(sim.shape)

Ahora, creamos un `DataFrame` para encontrar el top de los documentos más similares:

In [ ]:
sims = pd.DataFrame(data={"text": corpus, "sim": sim})
top5 = (
        sims
        .sort_values(by="sim", ascending=False)
        .head(6)
        )

Veamos los documentos más relevantes:

In [ ]:
for doc in top5.iloc[1:, 0]:
    print("=" * 50)
    print(doc)

Como podemos ver, estos documentos también hablan sobre hardware de computadores.

### **5.3. Términos por Tópico**
---

LDA se caracteriza por obtener mejores resultados en comparación con LSA. Esto se puede evidenciar en la matriz tópico-término aprendida, la cual podemos extraer con el atributo `components_` del modelo:

In [ ]:
components = lda.components_
print(components.shape)

Como podemos ver, nos muestra la importancia de cada una de las 500 palabras del vocabulario para cada uno de los 20 tópicos. Veamos el top 15 de las palabras más importantes por cada tópico:

In [ ]:
# Iteramos sobre cada tópico
for i, comp in enumerate(components):
    # Juntamos los términos con cada uno de los valores en la matriz V
    terms_comp = zip(vocab_bow, np.abs(comp))
    # Ordenamos los términos de acuerdo al resultado de LSA
    sorted_terms = sorted(
            terms_comp,
            key=lambda x: x[1],
            reverse=True
            )[:15]
    # Mostramos los términos más importantes en cada tópico
    print(
            "Tópico {}: {}".format(
                i,
                " ".join(list(map(lambda x:x[0], sorted_terms)))
                )
            )

Con esto podemos ver más tópicos clave como:

- **Tópico 3 y 15**: religión.
- **Tópico 4**: software.
- **Tópico 11 y 17**: hardware.
- **Tópico 12 y 16**: ciencia e investigación.
- **Tópico 13**: criptografía.
- **Tópico 14**: política.
- **Tópico 18**: deportes.

Vamos a generar una nube de palabras ponderada con la importancia de cada término en un tópico para visualizar mejor los tópicos:

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15, 7))
cont = 0
for i in range(4):
    for j in range(5):
        ax = axes[i, j]
        freqs = {
            term: abs(float(importance))
            for term, importance in zip(vocab_bow, components[cont])
        }
        wc = WordCloud(background_color="white").generate_from_frequencies(freqs)
        ax.imshow(wc)
        ax.axis("off")
        ax.set_title(f"Tópico {cont}")
        cont += 1
fig.show()

### **5.4. Importancia por Tópico**
---

El modelo LDA no nos da directamente las importancias de cada tópico dentro del corpus. No obstante, como estamos manejando probabilidades, estas importancias se pueden calcular directamente con la matriz de documento-tópico al promediar la contribución de los documentos. Es decir:

In [ ]:
topic_importances = features_lda.mean(axis=0)
print(topic_importances.shape)

Podemos generar una gráfica en forma de diagrama de barras para ver estas importancias:

In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(topic_importances.size), topic_importances)
ax.set_xticks(np.arange(topic_importances.size));
ax.set_xlabel("Tópico")
ax.set_ylabel("Importancia")
fig.show()

Por último, existe una herramienta conocida como **pyLDAvis**, la cual es una librería de Python para visualizar modelos de tópicos Latent Dirichlet Allocation (LDA) desarrollada por Carson Sievert. La biblioteca proporciona una interfaz web interactiva que permite explorar y comprender fácilmente los tópicos generados por un modelo LDA.

pyLDAvis genera una visualización interactiva que muestra los tópicos en un plano de coordenadas de dos dimensiones, donde cada punto representa un tópico y los puntos cercanos entre sí representan tópicos similares. Los tópicos se pueden explorar haciendo clic en ellos para ver las palabras más importantes y los documentos asociados.

También incluye un mapa de calor que muestra las palabras más relevantes para cada tópico, una nube de palabras que muestra la frecuencia de las palabras en los tópicos y un gráfico de barras que muestra la distribución de los tópicos en los documentos.

pyLDAvis es una herramienta útil para explorar y comprender los resultados de un modelo LDA, y puede ser utilizado para ajustar los parámetros del modelo y para seleccionar el número óptimo de tópicos.

Vamos a instalarla:

In [ ]:
!pip install pyLDAvis

Por problemas de compatibilidad, es necesario utilizar una version de `pandas < 2.0.0`. **No debe reiniciar el entorno de ejecución**

In [ ]:
!pip install pandas==1.5.3

Importamos la librería para su uso con `sklearn`:

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model as sklearn_lda

Habilitamos el uso de la librería en notebooks:

In [ ]:
pyLDAvis.enable_notebook()

Especificamos a la librería el vectorizador usado y el modelo LDA entrenado:

In [ ]:
bow.get_feature_names = bow.get_feature_names_out
ldavis_prepared = sklearn_lda.prepare(lda, features_bow, bow)

Podemos visualizar el tablero interactivo de LDA:

In [ ]:
display(ldavis_prepared)

También podemos exportarlo como un HTML para embeberlo, reutilizarlo o modificarlo:

In [ ]:
pyLDAvis.save_html(ldavis_prepared, "./ldavis_prepared.html")

## **Recursos Adicionales**
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este taller guiado:

- [Dimensionality reduction using truncated SVD (aka LSA)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).
- [Latent Dirichlet Allocation](https://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation).

## **Créditos**

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*